<a href="https://colab.research.google.com/github/Akash35678/BirdsongClassifier/blob/main/BirdSongProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import IPython.display as ipd
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
import os


# Navigate to the folder where your files are located
folder_path = '/content/drive/MyDrive/ExtendedBirdDataset/train'  # Replace with your folder's path
os.chdir(folder_path)

# List the files in the folder and store their names in an array
file_names = os.listdir()

# Print the list of file names
print(file_names)




['WrenBabbler', 'StreakedSpiderhunter', 'WedgeBilledWarbler', 'YellowBreastedWarbler', 'YellowBelliedFantail', 'YellowVentedFlowerpecker', 'StreakBreastedBabbler', 'WhiteTailedFlycatcher', 'YellowThroatedFulvetta', 'VernalHangingParrot', 'PlainFlowerpecker', 'ScarletBackedFlowerpecker', 'Pnoepyga pusilla', 'SichuanLeafWarbler', 'Stachyridopsis ambigua', 'PuffThroatedBabbler', 'SpotBreastedParrotbill', 'RedFacedLiocichla', 'PinStrippedBabbler', 'SlenderBilledBabbler', 'IndianCuckoo', 'JungleMyna', 'OrientalDollarbird', 'Leafbird', 'PaleBlueFlycatcher', 'PaleChinnedFlycatcher', 'LargeBlueFlycatcher', 'LineatedBarbet', "Jerdon'sLeafbird", 'LongBilledWrenBabbler', 'GreyBackedShrike', 'CollaredKingfisher', 'GreyPeacockPheasant', "Gould'sSunbird", 'HillPatridge', 'CitrineWagtail', 'GreyPlover', 'Halcyon coromanda', 'GreyThroatedMartin', 'CinnamonBittern', 'AsianBarredOwlet', 'BlackBrowedReedWarbler', 'AndamanDrongo', 'BuffThroatedWarbler', 'AsianPalmSwift', 'BlueWingedLeafbird', 'AsianEmeral

In [ ]:
import os
import librosa
import numpy as np

# Define paths to your audio data directories
train_data_dir = "/content/drive/MyDrive/ExtendedBirdDataset/train"

# Initialize lists to store audio data and labels
data = []
labels = []

# List of class names (bird species)
class_names = file_names
class_to_label = {class_name: i for i, class_name in enumerate(class_names)}


# Function to load and preprocess audio data
def load_and_preprocess_audio(audio_path, target_shape):
    # Load the audio file using Librosa
    signal, sr = librosa.load(audio_path, duration=10)  # Adjust duration as needed

    # Preprocess the audio data (you may need to extract features)
    # Example: Compute MFCC features from the audio
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=target_shape[0])

    # Pad or truncate the MFCC feature matrix to the target shape
    if mfccs.shape[1] < target_shape[1]:
        # Pad with zeros
        mfccs = np.pad(mfccs, ((0, 0), (0, target_shape[1] - mfccs.shape[1])), mode='constant')
    else:
        # Truncate
        mfccs = mfccs[:, :target_shape[1]]

    return mfccs

# Loop through class directories
for class_name in class_names:
    class_dir = os.path.join(train_data_dir, class_name)

    # Loop through audio files in the class directory
    for filename in os.listdir(class_dir):
        if filename.endswith(".wav"):
            # Load and preprocess the audio
            audio_path = os.path.join(class_dir, filename)
            audio_data = load_and_preprocess_audio(audio_path, target_shape=(13, 316))

            # Append the audio data and corresponding label
            data.append(audio_data)
            labels.append(class_to_label[class_name])

# Convert data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Verify the shape of the data array (should be of shape [num_samples, num_mfcc_features, num_frames])
print("Data shape:", data.shape)




Data shape: (925, 13, 316)


In [ ]:
data

array([[[-3.13851898e+02, -2.10653229e+02, -2.02836090e+02, ...,
         -3.31770050e+02, -3.31466278e+02, -3.31642487e+02],
        [ 9.60982285e+01,  8.98276138e+01,  7.81309280e+01, ...,
          3.29604607e+01,  3.54415932e+01,  3.21174355e+01],
        [ 2.33757095e+01,  2.12188416e+01,  1.73258476e+01, ...,
          2.44853325e+01,  2.54410973e+01,  2.50377827e+01],
        ...,
        [ 7.35777569e+00,  3.75514412e+00,  1.59428525e+00, ...,
          6.73910713e+00,  2.54890966e+00,  3.52627325e+00],
        [ 9.52860641e+00,  8.99256516e+00,  4.33318138e+00, ...,
         -4.25015241e-01, -3.63287735e+00, -8.96474171e+00],
        [ 7.42515850e+00,  6.32533979e+00,  5.70749474e+00, ...,
          8.72002983e+00,  1.03577118e+01,  4.10892582e+00]],

       [[-4.10014801e+02, -3.27602264e+02, -3.65323242e+02, ...,
         -4.69094360e+02, -4.70032837e+02, -4.68754211e+02],
        [ 1.15176735e+02,  1.16250580e+02,  1.15063782e+02, ...,
          8.65882263e+01,  8.67082520e

In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

model = models.Sequential([
    layers.Input(shape=(13, 316, 1)),  # Input shape should match the preprocessed data shape
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')  # Number of output classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Train the model
model.fit(data, labels, epochs=10, validation_split=0.2, batch_size=16)

# Save the trained model if needed
model.save("audio_classifier_model.h5")

NameError: ignored

In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras import models

# Define the path to your trained model
model_path = "audio_classifier_model.h5"  # Replace with the actual path to your model file

# Load the trained model
model = models.load_model(model_path)

# Define a function to preprocess an MP3 file
def preprocess_mp3(mp3_path, target_shape):
    # Load the MP3 file using Librosa
    signal, sr = librosa.load(mp3_path, duration=10)  # Adjust duration as needed

    # Preprocess the audio data (e.g., compute MFCC features)
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=target_shape[0])

    # Pad or truncate the MFCC feature matrix to the target shape
    if mfccs.shape[1] < target_shape[1]:
        # Pad with zeros
        mfccs = np.pad(mfccs, ((0, 0), (0, target_shape[1] - mfccs.shape[1])), mode='constant')
    else:
        # Truncate
        mfccs = mfccs[:, :target_shape[1]]

    # Reshape to match the model input shape (add a batch dimension)
    mfccs = np.expand_dims(mfccs, axis=-1)

    return mfccs

# Path to the MP3 file you want to test
mp3_file_path = "/content/XC472540 - Jungle Myna - Acridotheres fuscus.mp3"

# Preprocess the MP3 file
preprocessed_audio = preprocess_mp3(mp3_file_path, target_shape=(13, 316))

# Make predictions using the model
predictions = model.predict(np.array([preprocessed_audio]))

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Define class names
class_names=file_names

# Get the predicted class name
predicted_class_name = class_names[predicted_class_index]

# Print the result
print(f"Predicted class: {predicted_class_name}")

1/1 [==============================] - 0s 84ms/step
Predicted class: JungleMyna


In [ ]:
class_names


['WrenBabbler',
 'StreakedSpiderhunter',
 'WedgeBilledWarbler',
 'YellowBreastedWarbler',
 'YellowBelliedFantail',
 'YellowVentedFlowerpecker',
 'StreakBreastedBabbler',
 'WhiteTailedFlycatcher',
 'YellowThroatedFulvetta',
 'VernalHangingParrot',
 'PlainFlowerpecker',
 'ScarletBackedFlowerpecker',
 'Pnoepyga pusilla',
 'SichuanLeafWarbler',
 'Stachyridopsis ambigua',
 'PuffThroatedBabbler',
 'SpotBreastedParrotbill',
 'RedFacedLiocichla',
 'PinStrippedBabbler',
 'SlenderBilledBabbler',
 'IndianCuckoo',
 'JungleMyna',
 'OrientalDollarbird',
 'Leafbird',
 'PaleBlueFlycatcher',
 'PaleChinnedFlycatcher',
 'LargeBlueFlycatcher',
 'LineatedBarbet',
 "Jerdon'sLeafbird",
 'LongBilledWrenBabbler',
 'GreyBackedShrike',
 'CollaredKingfisher',
 'GreyPeacockPheasant',
 "Gould'sSunbird",
 'HillPatridge',
 'CitrineWagtail',
 'GreyPlover',
 'Halcyon coromanda',
 'GreyThroatedMartin',
 'CinnamonBittern',
 'AsianBarredOwlet',
 'BlackBrowedReedWarbler',
 'AndamanDrongo',
 'BuffThroatedWarbler',
 'AsianPa